# Training and testing of objection detection network
Based on Faster-RCNN and this guide: https://pytorch.org/tutorials/intermediate/torchvision_tutorial.html


In [1]:
# %%shell

#pip install torchvision
# # Install pycocotools, the version by default in Colab
# # has a bug fixed in https://github.com/cocodataset/cocoapi/pull/354
# pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

## Importing libraries and custom scripts


In [1]:
import sys
import os
import numpy as np
import torch
import torch.utils.data



## Dataset class with new __getitem__ function

In [2]:
import os
import numpy as np
import torch
import torch.utils.data
from PIL import Image


class CansDataset(torch.utils.data.Dataset):
    def __init__(self, root):
        self.root = root
        self.imgs = list(sorted(os.listdir(os.path.join(root,"video1/train/frames"))))
        self.bbox = list(sorted(os.listdir(os.path.join(root,"video1/train/boundingboxes"))))
    
    def __getitem__(self, idx):
        # load images and bboxes
        img_path = os.path.join(self.root, "video1/train/frames", self.imgs[idx])
        bbox_path = os.path.join(self.root, "video1/train/boundingboxes", self.bbox[idx])
        img = Image.open(img_path).convert("RGB")


        bbox = []
        label = []
        with open(bbox_path, 'r') as f:
            for line in f:
                line = line.split(" ")
                id = line[0] # class label, 0=beer, 1=cola
                id = 0 if id == 'beer' else 1
                xmin = int(line[1])
                ymin = int(line[2])
                xmax = int(line[3])
                ymax = int(line[4])
                bbox.append([xmin, ymin, xmax, ymax])
                label.append(id)
        bbox = torch.as_tensor(bbox, dtype=torch.float32)
        labels = torch.as_tensor(label, dtype=torch.float32) #torch.ones((num_objs, ), dtype=torch.float32)
        image_id = torch.tensor([idx])
        area = (bbox[:, 3] - bbox[:, 1]) * (bbox[:, 2] - bbox[:, 0])

        target = {}
        target["boxes"] = bbox
        target["labels"] = labels
        target["image_id"] = image_id
        target["area"] = area


        return img, target

    def __len__(self):
        return len(self.imgs)

Test class

In [3]:

#root = os.getcwd()+'data/'#os.getcwd() + '/..' + '/data/'
sys.path.append(os.getcwd() + "/.." + "/scripts")
print(os.getcwd())
os.chdir(os.getcwd()+"/..")

#print(root)
root = os.getcwd() + "/data"
dataset = CansDataset(root)


# img,target = dataset.__getitem__(907)
# print("img",img)
# print("target",target)

c:\Sync\Dokumenter\Universitet\Master\7_semester\02456_Deep_learning\02456-project\notebooks


## Adding pretrained model and modify numbers of classes

In [4]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator

# load a model pre-trained pre-trained on COCO
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

# replace the classifier with a new one, that has
# num_classes which is user-defined
num_classes = 2  # 1 class (person) + background
# get number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features
# replace the pre-trained head with a new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)


Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to C:\Users\joni_/.cache\torch\hub\checkpoints\fasterrcnn_resnet50_fpn_coco-258fb6c6.pth
100.0%


In [6]:
from engine import train_one_epoch, evaluate
import utils
import transforms as T


def get_transform(train):
    transforms = []
    # converts the image, a PIL image, into a PyTorch Tensor
    transforms.append(T.ToTensor())
    if train:
        # during training, randomly flip the training images
        # and ground-truth for data augmentation
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

In [7]:

sys.path.append(os.getcwd() + "/.." + "/scripts")
from engine import train_one_epoch, evaluate

import utils
import transforms as T
from typing import List, Tuple

data_loader = torch.utils.data.DataLoader(
    dataset, batch_size=2, shuffle=True, num_workers=4,
    collate_fn=utils.collate_fn
)
# For Training
images,targets = next(iter(data_loader))
images = list(image for image in images)
print(images)
#print(images[0].size[-2:])
#images = images[:4]
#targets = targets[:4]
targets = [{k: v for k, v in t.items()} for t in targets]
for img in images:
    print(img)
output = model(images,targets)   # Returns losses and detections
# # For inference
# model.eval()
# x = [torch.rand(3, 300, 400), torch.rand(3, 500, 400)]
# predictions = model(x)           # Returns predictions



RuntimeError: DataLoader worker (pid(s) 5032, 4940, 4692, 14828) exited unexpectedly

In [12]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
data_loader = torch.utils.data.DataLoader(
    dataset, batch_size=1, shuffle=True, num_workers=1,
    collate_fn=utils.collate_fn
)
# For Training
images,targets = next(iter(data_loader))
images = list(image for image in images)
targets = [{k: v for k, v in t.items()} for t in targets]
output = model(images,targets)   # Returns losses and detections
# For inference
model.eval()
x = [torch.rand(3, 300, 400), torch.rand(3, 500, 400)]
predictions = model(x)           # Returns predictions

RuntimeError: DataLoader worker (pid(s) 21488) exited unexpectedly

### To Do
Fix get item så den kører med Jonas' data\
Få vores data til at kører med den pretrænede model\
Test forward pass\
Benyt Holgers split til at træne med og opnå fuld model

